## Connect to DB :

In [1]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///student.db")
print(db.dialect)
print(db.get_usable_table_names())
db.run("SELECT * FROM Student LIMIT 10;")

sqlite
['STUDENT']


/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


"[('Krish', 'Data Science', 'A', 90), ('John', 'Data Science', 'B', 100), ('Mukesh', 'Data Science', 'A', 86), ('Jacob', 'DEVOPS', 'A', 50), ('Dipesh', 'DEVOPS', 'A', 35), ('Krish', 'Data Science', 'A', 90), ('John', 'Data Science', 'B', 100), ('Mukesh', 'Data Science', 'A', 86), ('Jacob', 'DEVOPS', 'A', 50), ('Dipesh', 'DEVOPS', 'A', 35)]"

In [121]:

# db = SQLDatabase.from_uri("sqlite:///chinook.db")
# print(db.dialect)
# print(db.get_usable_table_names())
# db.run("SELECT * FROM albums LIMIT 10;")

In [122]:

from langchain_community.llms import HuggingFaceEndpoint

# repo_id="meta-llama/Llama-2-70b-hf"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.2"
# repo_id = "mistralai/Mistral-7B-Instruct-v0.3"

repo_id="meta-llama/Meta-Llama-3-8B-Instruct"


llm = HuggingFaceEndpoint(repo_id = repo_id, huggingfacehub_api_token = API_KEY, return_full_text=True)

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/sayakmisra/.cache/huggingface/token
Login successful


## Using chains and tools:

In [3]:
# from langchain_community.llms import Ollama

# llm = Ollama(model="llama3.1:8b-instruct-q4_0")
from langchain_groq import ChatGroq
llm=ChatGroq(groq_api_key='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',model_name="llama3-70b-8192",streaming=True)

In [124]:
def parse_final_answer(output: str) -> str:
    return output.split('SQLQuery:')[1]

## Implementing the chain (create_sql_query_chain):

In [125]:
from langchain.chains import create_sql_query_chain

chain = create_sql_query_chain(llm, db)
response = chain.invoke({"question": "How many students are there"})
# final = response.rsplit('SQLQuery:')[2]
response

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method 

'Here is the answer:\n\nQuestion: How many students are there\nSQLQuery: SELECT COUNT("NAME") FROM "STUDENT"'

In [126]:
chain_new = create_sql_query_chain(llm, db) | parse_final_answer
# prompt.pretty_print()

In [127]:
query = "Show the students in section B"
query = chain_new.invoke(
    {
        "question": query
    }
)
print(query)

 SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = 'B' LIMIT 5;


/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method 

In [128]:
chain.get_prompts()[0].pretty_print()

You are a SQLite expert. Given an input question, first create a syntactically correct SQLite query to run, then look at the results of the query and return the answer to the input question.
Unless the user specifies in the question a specific number of examples to obtain, query for at most 5 results using the LIMIT clause as per SQLite. You can order the results to return the most informative data in the database.
Never query for all columns from a table. You must query only the columns that are needed to answer the question. Wrap each column name in double quotes (") to denote them as delimited identifiers.
Pay attention to use only the column names you can see in the tables below. Be careful to not query for columns that do not exist. Also, pay attention to which column is in which table.
Pay attention to use date('now') function to get the current date, if the question involves "today".

Use the following format:

Question: Question here
SQLQuery: SQL Query to run
SQLResult: Result

In [129]:
# db.run(final)

## Implementing the tool (QuerySQLDataBaseTool):

In [130]:
from langchain_community.tools.sql_database.tool import QuerySQLDataBaseTool

execute_query = QuerySQLDataBaseTool(db=db)
write_query = create_sql_query_chain(llm, db)
chain = write_query | parse_final_answer | execute_query
chain.invoke({"question": query})

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method 

"[('John',), ('John',)]"

In [131]:
test_chain = write_query | parse_final_answer
test_chain.invoke({"question": query})

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method 

' SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;'

In [132]:
from operator import itemgetter

from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough

answer_prompt = PromptTemplate.from_template(
    """Given the following user question, corresponding SQL query, and SQL result, answer the user question.

Question: {question}
SQL Query: {query}
SQL Result: {result}
Answer: """
)

chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query") | execute_query
    )
    | answer_prompt
    | llm
    | StrOutputParser()
)
test_chain = write_query | parse_final_answer  | execute_query
test_chain.invoke({"question": query})

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method 

"[('John',), ('John',)]"

In [110]:
chain = (
    RunnablePassthrough.assign(query=write_query).assign(
        result=itemgetter("query")))
chain.invoke({"question": query})

{'question': ' SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;',
 'query': 'Here is the answer:\n\nQuestion: SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;\nSQLQuery: SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;',
 'result': 'Here is the answer:\n\nQuestion: SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;\nSQLQuery: SELECT "NAME" FROM "STUDENT" WHERE "SECTION" = \'B\' LIMIT 5;'}

In [58]:
from langchain_core.messages import SystemMessage

SQL_PREFIX = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the below tools. Only use the information returned by the below tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

To start you should ALWAYS look at the tables in the database to see what you can query.
Do NOT skip this step.
Then you should query the schema of the most relevant tables."""

system_message = SystemMessage(content=SQL_PREFIX)

In [1]:
# %pip install -U langchain-community langgraph langgraph-checkpoint-sqlite

  Using cached langchain_community-0.2.17-py3-none-any.whl.metadata (2.7 kB)
ERROR: Ignored the following versions that require a different python version: 0.0.10 Requires-Python >=3.9.0,<4.0; 0.0.11 Requires-Python >=3.9.0,<4.0; 0.0.12 Requires-Python >=3.9.0,<4.0; 0.0.13 Requires-Python >=3.9.0,<4.0; 0.0.14 Requires-Python >=3.9.0,<4.0; 0.0.15 Requires-Python >=3.9.0,<4.0; 0.0.16 Requires-Python >=3.9.0,<4.0; 0.0.17 Requires-Python >=3.9.0,<4.0; 0.0.18 Requires-Python >=3.9.0,<4.0; 0.0.19 Requires-Python >=3.9.0,<4.0; 0.0.20 Requires-Python >=3.9.0,<4.0; 0.0.21 Requires-Python >=3.9.0,<4.0; 0.0.22 Requires-Python >=3.9.0,<4.0; 0.0.23 Requires-Python >=3.9.0,<4.0; 0.0.24 Requires-Python >=3.9.0,<4.0; 0.0.25 Requires-Python >=3.9.0,<4.0; 0.0.26 Requires-Python >=3.9.0,<4.0; 0.0.27 Requires-Python >=3.9.0,<4.0; 0.0.28 Requires-Python >=3.9.0,<4.0; 0.0.29 Requires-Python <4.0,>=3.9.0; 0.0.30 Requires-Python <4.0,>=3.9.0; 0.0.31 Requires-Python <4.0,>=3.9.0; 0.0.32 Requires-Python <4.0,>=

In [62]:
# %%capture --no-stderr
# %pip install --upgrade --quiet langgraph

## Using SQL-Agents :

In [7]:
from langchain.agents import create_sql_agent, create_react_agent
from langchain.sql_database import SQLDatabase
from langchain.agents.agent_types import AgentType
from langchain.agents.agent_toolkits import SQLDatabaseToolkit
from sqlalchemy import create_engine
from langchain_groq import ChatGroq

In [119]:
from langchain_community.agent_toolkits import SQLDatabaseToolkit

toolkit = SQLDatabaseToolkit(db=db, llm=llm)

tools = toolkit.get_tools()

# tools

## Using create_react_agent :

In [118]:
from langchain_core.messages import HumanMessage
from langchain import hub
# from langgraph.checkpoint.memory import MemorySaver
# from langgraph.prebuilt import create_react_agent
from langchain.agents import AgentExecutor, create_react_agent
# from langchain.agents.react.agent import create_react_agent

prompt = hub.pull("hwchase17/react")

agent = create_react_agent(llm, tools, prompt)
agent_executor = AgentExecutor(agent=agent, tools=tools)
user_query = 'show students in section A?'
user_query = 'how many unique students are there?'
agent_executor.invoke({"input": user_query})

/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/langchain/hub.py:86: DeprecationWarning: The `langchainhub sdk` is deprecated.
Please use the `langsmith sdk` instead:
  pip install langsmith
Use the `pull_prompt` method.
  res_dict = client.pull_repo(owner_repo_commit)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.8/migration/
  warnings.warn('The `dict` method is deprecated; use `model_dump` instead.', category=PydanticDeprecatedSince20)
/Users/sayakmisra/Projects/email_generator/venv/lib/python3.8/site-packages/pydantic/main.py:1087: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at htt

{'input': 'how many unique students are there?',
 'output': "There are 3 unique students.\n\nNote: The observation `[(5,)]` suggests that the query returned a count of 5, but based on the sample data provided in the schema, there are only 3 unique students (Krish, John, and Mukesh). It's possible that there are more students in the actual database, but based on the provided information, the correct answer is 3."}

## Using create_sql_agent :

In [8]:
llm=ChatGroq(groq_api_key='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',model_name="llama3-70b-8192",streaming=True)

In [9]:
## toolkit
toolkit=SQLDatabaseToolkit(db=db,llm=llm)

agent=create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

In [10]:

user_query = 'how many unique students are there in section A?'
user_query = 'show students with score more than 30?'
# user_query = 'show students in section A?'
# user_query = 'show all students?'
# response=agent.run(user_query)
response=agent.invoke(user_query)
response



> Entering new SQL Agent Executor chain...
Thought: I need to find the table that contains student information and scores. 

Action: sql_db_list_tables
Action Input: STUDENTThought: I have the table name, which is STUDENT. Now I need to know the schema of the STUDENT table to find the correct column names for student information and scores.

Action: sql_db_schema
Action Input: STUDENT
CREATE TABLE "STUDENT" (
	"NAME" VARCHAR(25), 
	"CLASS" VARCHAR(25), 
	"SECTION" VARCHAR(25), 
	"MARKS" INTEGER
)

/*
3 rows from STUDENT table:
NAME	CLASS	SECTION	MARKS
Krish	Data Science	A	90
John	Data Science	B	100
Mukesh	Data Science	A	86
*/Thought: I have the schema of the STUDENT table. I see that the column names are NAME, CLASS, SECTION, and MARKS. I need to write a query to select students with MARKS more than 30.

Action: sql_db_query_checker
Action Input: SELECT NAME FROM STUDENT WHERE MARKS > 30SELECT NAME FROM STUDENT WHERE MARKS > 30Thought: The query looks good, I'll execute it to get the

{'input': 'show students with score more than 30?',
 'output': 'The students with scores more than 30 are Krish, John, Mukesh, Jacob, and Dipesh.'}

## Using a dynamic few-shot prompt :

In [60]:
examples = [
    {"input": "List all artists.", "query": "SELECT * FROM Artist;"},
    {
        "input": "Find all albums for the artist 'AC/DC'.",
        "query": "SELECT * FROM Album WHERE ArtistId = (SELECT ArtistId FROM Artist WHERE Name = 'AC/DC');",
    },
    {
        "input": "List all tracks in the 'Rock' genre.",
        "query": "SELECT * FROM Track WHERE GenreId = (SELECT GenreId FROM Genre WHERE Name = 'Rock');",
    },
    {
        "input": "Find the total duration of all tracks.",
        "query": "SELECT SUM(Milliseconds) FROM Track;",
    },
    {
        "input": "List all customers from Canada.",
        "query": "SELECT * FROM Customer WHERE Country = 'Canada';",
    },
    {
        "input": "How many tracks are there in the album with ID 5?",
        "query": "SELECT COUNT(*) FROM Track WHERE AlbumId = 5;",
    },
    {
        "input": "Find the total number of invoices.",
        "query": "SELECT COUNT(*) FROM Invoice;",
    },
    {
        "input": "List all tracks that are longer than 5 minutes.",
        "query": "SELECT * FROM Track WHERE Milliseconds > 300000;",
    },
    {
        "input": "Who are the top 5 customers by total purchase?",
        "query": "SELECT CustomerId, SUM(Total) AS TotalPurchase FROM Invoice GROUP BY CustomerId ORDER BY TotalPurchase DESC LIMIT 5;",
    },
    {
        "input": "Which albums are from the year 2000?",
        "query": "SELECT * FROM Album WHERE strftime('%Y', ReleaseDate) = '2000';",
    },
    {
        "input": "How many employees are there",
        "query": 'SELECT COUNT(*) FROM "Employee"',
    },
]

In [35]:
# # from langchain_core.prompts import FewShotPromptTemplate, PromptTemplate
# from langchain_core.prompts import (
#     ChatPromptTemplate,
#     FewShotPromptTemplate,
#     MessagesPlaceholder,
#     PromptTemplate,
#     SystemMessagePromptTemplate,
# )

# example_prompt = PromptTemplate.from_template("User input: {input}\nSQL query: {query}")
# few_shot_prompt = FewShotPromptTemplate(
#     examples=examples[:5],
#     example_prompt=example_prompt,
#     prefix="You are a SQLite expert. Given an input question, create a syntactically correct SQLite query to run. Unless otherwise specificed, do not return more than {top_k} rows. \n\nBelow are a number of examples of questions and their corresponding SQL queries.",
#     suffix="User input: {input}\nSQL query: ",
#     input_variables=["input", "top_k"],
# )

In [61]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    examples=examples[:5],
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)

In [58]:
# from langchain_core.prompts import (
#     ChatPromptTemplate,
#     FewShotChatMessagePromptTemplate,
#     PromptTemplate,
# )

# from langchain.agents.mrkl import prompt as react_prompt

# examples = [
#   {"input": "List all the characters in Anna Karenina",
#    "query": "SELECT DISTINCT characterNames FROM novels WHERE novelName = 'Anna Karenina'"},
# ]

# system_prefix = """
#     You are an agent designed to interact with a SQL database.
#     Given an input question, create a syntactically correct query to run, then look at the results of the query and 
#     return the answer. Unless the user specifies a specific number of examples they wish to obtain.
#     You can order the results by a relevant column to return the most interesting examples in the database.
#     Never query for all the columns from a specific table, only ask for the relevant columns given the question.
#     You have access to tools for interacting with the database.
#     Only use the given tools. Only use the information returned by the tools to construct your final answer.
#     You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query 
#     and try again.

#     DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

#     If the question does not seem related to the database, just return "I don't know" as the answer.
# """

# basic_suffix = """
#     Begin!

#     Question: {input}
#     Thought: I should look at the tables in the database to see what I can query. Then I should query the schema of the most relevant tables.
#     {agent_scratchpad}

# """

# example_prompt = ChatPromptTemplate.from_messages(
#     messages=[
#         ('human', "{input}"),
#         ('ai', "{query}")
#     ]
# )

# few_shot_prompt = FewShotChatMessagePromptTemplate(
#     examples=examples,
#     example_prompt=example_prompt,
#     input_variables=["input",
#                      "agent_scratchpad"],
# )

# format_instructions = f"{react_prompt.FORMAT_INSTRUCTIONS}\n " \
#                       f"Here are some examples of user inputs and " \
#                       f"their corresponding SQL queries:\n"

# template = "\n\n".join(
#     [
#         system_prefix,
#         "{tools}",
#         format_instructions,
#         few_shot_prompt.format(),
#         basic_suffix
#     ]
# )

# full_prompt = PromptTemplate.from_template(template=template)
# full_prompt

In [59]:
# # Example formatted prompt
# prompt_val = full_prompt.invoke(
#     {
#         "input": "How many arists are there",
#         "tools": tools,
#         "tool_names": '',
#         "dialect": "SQLite",
#         "agent_scratchpad": [],
#     }
# )
# print(prompt_val.to_string())

In [46]:
# print(few_shot_prompt.format(input="How many students are there?", top_k=3))

In [62]:
full_prompt = ChatPromptTemplate.from_messages(
    [
        SystemMessagePromptTemplate(prompt=few_shot_prompt),
        ("human", "{input}"),
        MessagesPlaceholder("agent_scratchpad"),
    ]
)

In [63]:
# Example formatted prompt
prompt_val = full_prompt.invoke(
    {
        "input": "How many arists are there",
        "top_k": 5,
        "dialect": "SQLite",
        "agent_scratchpad": [],
    }
)
print(prompt_val.to_string())

System: You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct SQLite query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most 5 results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't 

In [64]:
agent = create_sql_agent(
    llm=llm,
    db=db,
    prompt=full_prompt,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
)

ValueError: Prompt missing required variables: {'tools', 'tool_names'}

In [42]:
new_agent=create_sql_agent(
    llm=llm,
    db=db,
    # toolkit=toolkit,
    prompt=full_prompt,
    verbose=True,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION
)

ValueError: Prompt missing required variables: {'tools', 'tool_names'}

In [ ]:
chain = create_sql_query_chain(llm, db, prompt)
chain.invoke({"question": "how many artists are there?"})

In [ ]:
from langchain_core.prompts import (
    ChatPromptTemplate,
    FewShotPromptTemplate,
    MessagesPlaceholder,
    PromptTemplate,
    SystemMessagePromptTemplate,
)

system_prefix = """You are an agent designed to interact with a SQL database.
Given an input question, create a syntactically correct {dialect} query to run, then look at the results of the query and return the answer.
Unless the user specifies a specific number of examples they wish to obtain, always limit your query to at most {top_k} results.
You can order the results by a relevant column to return the most interesting examples in the database.
Never query for all the columns from a specific table, only ask for the relevant columns given the question.
You have access to tools for interacting with the database.
Only use the given tools. Only use the information returned by the tools to construct your final answer.
You MUST double check your query before executing it. If you get an error while executing a query, rewrite the query and try again.

DO NOT make any DML statements (INSERT, UPDATE, DELETE, DROP etc.) to the database.

If the question does not seem related to the database, just return "I don't know" as the answer.

Here are some examples of user inputs and their corresponding SQL queries:"""

few_shot_prompt = FewShotPromptTemplate(
    example_selector=example_selector,
    example_prompt=PromptTemplate.from_template(
        "User input: {input}\nSQL query: {query}"
    ),
    input_variables=["input", "dialect", "top_k"],
    prefix=system_prefix,
    suffix="",
)